In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
import tensorflow as tf
%load_ext tensorboard

In [2]:
data_path_and_labels = pd.read_csv("../mapped_data/camel-1.6.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/camel-1.6/camel-camel-1.6.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [3]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/camel-1.6/camel-camel-1.6.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [4]:
data_path_and_labels['data_path'] = data_path_and_labels['data_path'].str.replace('.embed','')
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/camel-1.6/camel-camel-1.6.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent'

In [5]:
words = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = f.read().splitlines()
    words.append(lines)

In [6]:
d = pd.read_csv("../mapped_data/camel-1.6.csv")
d['data_path'][0]

'dataset/promise/defect data/camel-1.6/camel-camel-1.6.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [7]:
d['data_path'] = '../' + d['data_path']
d['data_path'][0]

'../dataset/promise/defect data/camel-1.6/camel-camel-1.6.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [8]:
d

,data_path,bugs
0,../dataset/promise/defect data/camel-1.6/camel...,0
1,../dataset/promise/defect data/camel-1.6/camel...,0
2,../dataset/promise/defect data/camel-1.6/camel...,0
3,../dataset/promise/defect data/camel-1.6/camel...,0
4,../dataset/promise/defect data/camel-1.6/camel...,0
...,...,...
929,../dataset/promise/defect data/camel-1.6/camel...,0
930,../dataset/promise/defect data/camel-1.6/camel...,4
931,../dataset/promise/defect data/camel-1.6/camel...,3
932,../dataset/promise/defect data/camel-1.6/camel...,1


In [9]:
train, test = train_test_split(d, test_size=0.2)

In [10]:
test_x = []
for index, row in test.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    test_x.append(lines)

In [11]:
train_x = []
for index, row in train.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    train_x.append(lines)

In [12]:
vocab = float(max(np.max(train_x), np.max(test_x)))
vocab

7770.0

In [13]:
train_y = train[' bugs'].to_numpy()
test_y = test[' bugs'].to_numpy()

In [14]:
train_y

array([ 0,  0,  0,  2,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
        2, 18,  0,  0,  0,  4,  3,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        1,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,
        0,  0,  3,  0,  0,  0,  1,  1,  0,  0,  0,  1,  2,  0,  0,  0,  0,
        0,  1,  1,  0,  0,  0,  1,  0,  3,  0,  0,  0,  0,  0,  1,  0,  0,
        0,  2,  0,  0,  5,  0,  1,  0,  0,  1,  0,  0,  0,  1,  2,  0,  0,
        0,  1,  0,  0,  0,  4,  0,  0,  0,  2,  1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  5,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  4,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 10,
        1,  2,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  1,
        0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,
        0,  0,  0,  1,  3

In [15]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [16]:
train_x.shape

(747, 2094)

In [17]:
# define the model
model = Sequential()
model.add(tf.keras.layers.Embedding(10671, 30, input_length=train_x.shape[1]))
model.add(tf.keras.layers.Flatten())
model.add(Dense(1, activation='relu'))
# compile the model
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [18]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
24/24 [==============================] - 1s 10ms/step - loss: 3.8786 - mse: 3.8786 - mae: 0.9290 - root_mean_squared_error: 1.9694 - mean_squared_logarithmic_error: 0.3625
Epoch 2/100
24/24 [==============================] - 0s 9ms/step - loss: 3.7668 - mse: 3.7668 - mae: 0.8189 - root_mean_squared_error: 1.9408 - mean_squared_logarithmic_error: 0.2966
Epoch 3/100
24/24 [==============================] - 0s 9ms/step - loss: 3.6278 - mse: 3.6278 - mae: 0.8530 - root_mean_squared_error: 1.9047 - mean_squared_logarithmic_error: 0.2831
Epoch 4/100
24/24 [==============================] - 0s 10ms/step - loss: 3.5723 - mse: 3.5723 - mae: 0.8595 - root_mean_squared_error: 1.8901 - mean_squared_logarithmic_error: 0.2863
Epoch 5/100
24/24 [==============================] - 0s 9ms/step - loss: 3.4990 - mse: 3.4990 - mae: 0.8215 - root_mean_squared_error: 1.8705 - mean_squared_logarithmic_error: 0.2637
Epoch 6/100
24/24 [==============================] - 0s 9ms/step - loss: 3.4229 - m

24/24 [==============================] - 0s 10ms/step - loss: 1.3517 - mse: 1.3517 - mae: 0.4432 - root_mean_squared_error: 1.1626 - mean_squared_logarithmic_error: 0.1090
Epoch 45/100
24/24 [==============================] - 0s 9ms/step - loss: 1.3112 - mse: 1.3112 - mae: 0.3618 - root_mean_squared_error: 1.1451 - mean_squared_logarithmic_error: 0.0839
Epoch 46/100
24/24 [==============================] - 0s 9ms/step - loss: 1.2731 - mse: 1.2731 - mae: 0.3630 - root_mean_squared_error: 1.1283 - mean_squared_logarithmic_error: 0.0846
Epoch 47/100
24/24 [==============================] - 0s 9ms/step - loss: 1.2443 - mse: 1.2443 - mae: 0.3946 - root_mean_squared_error: 1.1155 - mean_squared_logarithmic_error: 0.0933
Epoch 48/100
24/24 [==============================] - 0s 9ms/step - loss: 1.2109 - mse: 1.2109 - mae: 0.3589 - root_mean_squared_error: 1.1004 - mean_squared_logarithmic_error: 0.0831
Epoch 49/100
24/24 [==============================] - 0s 9ms/step - loss: 1.1809 - mse: 1.18

24/24 [==============================] - 0s 9ms/step - loss: 0.5311 - mse: 0.5311 - mae: 0.1679 - root_mean_squared_error: 0.7287 - mean_squared_logarithmic_error: 0.0314
Epoch 89/100
24/24 [==============================] - 0s 9ms/step - loss: 0.5310 - mse: 0.5310 - mae: 0.1453 - root_mean_squared_error: 0.7287 - mean_squared_logarithmic_error: 0.0258
Epoch 90/100
24/24 [==============================] - 0s 9ms/step - loss: 0.5164 - mse: 0.5164 - mae: 0.1343 - root_mean_squared_error: 0.7186 - mean_squared_logarithmic_error: 0.0240
Epoch 91/100
24/24 [==============================] - 0s 9ms/step - loss: 0.5001 - mse: 0.5001 - mae: 0.1303 - root_mean_squared_error: 0.7071 - mean_squared_logarithmic_error: 0.0240
Epoch 92/100
24/24 [==============================] - 0s 9ms/step - loss: 0.4888 - mse: 0.4888 - mae: 0.1357 - root_mean_squared_error: 0.6991 - mean_squared_logarithmic_error: 0.0251
Epoch 93/100
24/24 [==============================] - 0s 9ms/step - loss: 0.4800 - mse: 0.480

In [19]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 21467), started 0:34:12 ago. (Use '!kill 21467' to kill it.)

In [20]:
score = model.evaluate(test_x, test_y)
score

6/6 [==============================] - 0s 3ms/step - loss: 1.3786 - mse: 1.3786 - mae: 0.6512 - root_mean_squared_error: 1.1742 - mean_squared_logarithmic_error: 0.2536


[1.3786487579345703,
 1.3786487579345703,
 0.6511659026145935,
 1.1741586923599243,
 0.2536270320415497]

In [21]:
data_path_and_labels = pd.read_csv("../mapped_data/camel-1.6.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/camel-1.6/camel-camel-1.6.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [22]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/camel-1.6/camel-camel-1.6.0/components/camel-irc/src/main/java/org/apache/camel/component/irc/IrcExchange.java.restrictedcontent.embed'

In [23]:
embeddings = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    embeddings.append(lines)

In [25]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093
0,990.0,984.0,2612.0,990.0,2986.0,252.0,5920.0,700.0,2612.0,984.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,992.0,1971.0,220.0,1971.0,1154.0,1115.0,4378.0,992.0,911.0,2937.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,986.0,1971.0,7274.0,1971.0,4872.0,1971.0,5917.0,1971.0,5659.0,1971.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,991.0,1971.0,5509.0,1971.0,7274.0,915.0,7652.0,1971.0,7714.0,1971.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,984.0,3764.0,1411.0,3669.0,990.0,5486.0,991.0,1971.0,7575.0,4422.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,32.0,3226.0,1529.0,694.0,3748.0,1971.0,6396.0,7501.0,3139.0,3748.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,756.0,3814.0,708.0,3748.0,1971.0,756.0,5281.0,756.0,6396.0,3139.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
931,694.0,708.0,617.0,6396.0,708.0,617.0,3632.0,1411.0,3669.0,617.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
932,1880.0,7102.0,708.0,3748.0,1971.0,1880.0,5281.0,1880.0,6396.0,3139.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
data = pd.concat([data_path_and_labels, df], axis = 1)
data = data.drop('data_path', 1)
data

,bugs,0,1,2,3,4,5,6,7,8,...,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093
0,0,990.0,984.0,2612.0,990.0,2986.0,252.0,5920.0,700.0,2612.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,992.0,1971.0,220.0,1971.0,1154.0,1115.0,4378.0,992.0,911.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,986.0,1971.0,7274.0,1971.0,4872.0,1971.0,5917.0,1971.0,5659.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,991.0,1971.0,5509.0,1971.0,7274.0,915.0,7652.0,1971.0,7714.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,984.0,3764.0,1411.0,3669.0,990.0,5486.0,991.0,1971.0,7575.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929,0,32.0,3226.0,1529.0,694.0,3748.0,1971.0,6396.0,7501.0,3139.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
930,4,756.0,3814.0,708.0,3748.0,1971.0,756.0,5281.0,756.0,6396.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
931,3,694.0,708.0,617.0,6396.0,708.0,617.0,3632.0,1411.0,3669.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
932,1,1880.0,7102.0,708.0,3748.0,1971.0,1880.0,5281.0,1880.0,6396.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
train, test = train_test_split(data, test_size=0.2)

In [28]:
train

,bugs,0,1,2,3,4,5,6,7,8,...,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093
118,0,796.0,1154.0,1115.0,4378.0,796.0,796.0,2932.0,2638.0,2816.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275,0,1435.0,1154.0,1115.0,4378.0,1435.0,4056.0,321.0,5596.0,1971.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
677,4,304.0,1134.0,321.0,3668.0,304.0,3668.0,1134.0,321.0,6082.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,0,1030.0,1971.0,1358.0,3123.0,646.0,534.0,7622.0,1971.0,6238.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
268,0,1039.0,1038.0,2877.0,2638.0,6580.0,1039.0,2877.0,1038.0,7622.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,0,1548.0,3230.0,1544.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
86,0,1820.0,1971.0,5281.0,1820.0,5281.0,1971.0,5281.0,5281.0,3226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
149,0,48.0,1154.0,1115.0,4378.0,48.0,1541.0,6064.0,951.0,6192.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
253,0,1399.0,4141.0,583.0,3532.0,651.0,6589.0,4662.0,4434.0,7515.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
test

,bugs,0,1,2,3,4,5,6,7,8,...,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093
339,0,872.0,872.0,4950.0,4950.0,4475.0,2285.0,4475.0,2285.0,4475.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
237,0,1504.0,1157.0,1115.0,4356.0,1504.0,1504.0,6931.0,5486.0,1223.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
129,0,1940.0,1971.0,2124.0,1971.0,6147.0,1037.0,5236.0,1940.0,3571.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
346,0,293.0,357.0,1971.0,863.0,2970.0,357.0,1971.0,863.0,293.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
622,0,1947.0,2194.0,4966.0,2198.0,5833.0,3307.0,2193.0,5809.0,1443.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,0,2212.0,1583.0,6389.0,3226.0,1529.0,694.0,3748.0,1971.0,2206.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
771,2,1247.0,2150.0,2150.0,694.0,3669.0,1411.0,2621.0,1971.0,5494.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
788,1,575.0,1128.0,3413.0,575.0,3413.0,1128.0,3413.0,3413.0,5739.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
928,0,4851.0,708.0,694.0,4856.0,1971.0,6396.0,4852.0,4856.0,3226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
Y_train = train[' bugs']
Y_test = test[' bugs']

In [31]:
train_y = Y_train.to_numpy()
test_y = Y_test.to_numpy()

In [32]:
train = train.drop(' bugs', axis=1)
test = test.drop(' bugs', axis=1)

In [33]:
X_train = train.values
X_test = test.values

In [34]:
train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [41]:
# Designing and initializing the model.
model = Sequential()
model.add(SimpleRNN(100, input_shape = (1,20), dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(80, dropout = 0.2, return_sequences=True))
model.add(SimpleRNN(60, dropout = 0.2, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [42]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100)

Epoch 1/100
24/24 [==============================] - 2s 5ms/step - loss: 2.9547 - mse: 2.9547 - mae: 0.5488 - root_mean_squared_error: 1.7189 - mean_squared_logarithmic_error: 0.2693
Epoch 2/100
24/24 [==============================] - 0s 4ms/step - loss: 2.9689 - mse: 2.9689 - mae: 0.5235 - root_mean_squared_error: 1.7230 - mean_squared_logarithmic_error: 0.2634
Epoch 3/100
24/24 [==============================] - 0s 4ms/step - loss: 2.9807 - mse: 2.9807 - mae: 0.5519 - root_mean_squared_error: 1.7265 - mean_squared_logarithmic_error: 0.2787
Epoch 4/100
24/24 [==============================] - 0s 4ms/step - loss: 3.0401 - mse: 3.0401 - mae: 0.5428 - root_mean_squared_error: 1.7436 - mean_squared_logarithmic_error: 0.2801
Epoch 5/100
24/24 [==============================] - 0s 3ms/step - loss: 2.9803 - mse: 2.9803 - mae: 0.5359 - root_mean_squared_error: 1.7264 - mean_squared_logarithmic_error: 0.2683
Epoch 6/100
24/24 [==============================] - 0s 3ms/step - loss: 3.0445 - mse

24/24 [==============================] - 0s 5ms/step - loss: 2.9176 - mse: 2.9176 - mae: 0.5487 - root_mean_squared_error: 1.7081 - mean_squared_logarithmic_error: 0.2617
Epoch 46/100
24/24 [==============================] - 0s 4ms/step - loss: 2.8992 - mse: 2.8992 - mae: 0.5556 - root_mean_squared_error: 1.7027 - mean_squared_logarithmic_error: 0.2606
Epoch 47/100
24/24 [==============================] - 0s 3ms/step - loss: 2.9777 - mse: 2.9777 - mae: 0.5828 - root_mean_squared_error: 1.7256 - mean_squared_logarithmic_error: 0.2832
Epoch 48/100
24/24 [==============================] - 0s 3ms/step - loss: 2.9026 - mse: 2.9026 - mae: 0.5617 - root_mean_squared_error: 1.7037 - mean_squared_logarithmic_error: 0.2588
Epoch 49/100
24/24 [==============================] - 0s 3ms/step - loss: 2.8812 - mse: 2.8812 - mae: 0.5511 - root_mean_squared_error: 1.6974 - mean_squared_logarithmic_error: 0.2542
Epoch 50/100
24/24 [==============================] - 0s 3ms/step - loss: 2.9051 - mse: 2.905

24/24 [==============================] - 0s 3ms/step - loss: 2.8479 - mse: 2.8479 - mae: 0.5955 - root_mean_squared_error: 1.6876 - mean_squared_logarithmic_error: 0.2601
Epoch 90/100
24/24 [==============================] - 0s 3ms/step - loss: 2.7914 - mse: 2.7914 - mae: 0.6229 - root_mean_squared_error: 1.6707 - mean_squared_logarithmic_error: 0.2669
Epoch 91/100
24/24 [==============================] - 0s 4ms/step - loss: 2.8287 - mse: 2.8287 - mae: 0.6210 - root_mean_squared_error: 1.6819 - mean_squared_logarithmic_error: 0.2721
Epoch 92/100
24/24 [==============================] - 0s 3ms/step - loss: 2.7717 - mse: 2.7717 - mae: 0.5936 - root_mean_squared_error: 1.6648 - mean_squared_logarithmic_error: 0.2518
Epoch 93/100
24/24 [==============================] - 0s 3ms/step - loss: 2.7898 - mse: 2.7898 - mae: 0.5997 - root_mean_squared_error: 1.6703 - mean_squared_logarithmic_error: 0.2594
Epoch 94/100
24/24 [==============================] - 0s 4ms/step - loss: 2.8510 - mse: 2.851

In [43]:
score = model.evaluate(test_x, test_y)
score

6/6 [==============================] - 0s 2ms/step - loss: 5.2256 - mse: 5.2256 - mae: 0.7541 - root_mean_squared_error: 2.2859 - mean_squared_logarithmic_error: 0.3139


[5.225556373596191,
 5.225556373596191,
 0.7540741562843323,
 2.285947561264038,
 0.31393346190452576]